# San Diego Vehicle Pull-Over Investigation

## Police Pull-Over Dataset Investigation 
The first thing is to spot the suspecious data entry from the database

In [1]:
import os
import glob

In [2]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
data_list = glob.glob('./vehicle_stops_20*_datasd.csv')
print(data_list)

['./vehicle_stops_2014_datasd.csv', './vehicle_stops_2016_datasd.csv', './vehicle_stops_2015_datasd.csv', './vehicle_stops_2017_datasd.csv']


In [4]:
df_list = [pd.read_csv(i) for i in data_list]
sd_stop = pd.concat(df_list, ignore_index=True)

/Users/macyeh/anaconda3/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [5]:
sd_stop.head()

,Unnamed: 15,Unnamed: as16,arrested,contraband_found,obtained_consent,property_seized,sd_resident,searched,service_area,stop_cause,stop_date,stop_id,stop_time,subject_age,subject_race,subject_sex,timestamp
0,NaN,NaN,N,N,N,N,Y,N,110,Moving Violation,2014-01-01,1044975,1:25,24,W,M,2014-01-01 01:25:00
1,NaN,NaN,N,NaN,NaN,NaN,Y,N,320,Moving Violation,2014-01-01,1044976,5:47,42,W,M,2014-01-01 05:47:00
2,NaN,NaN,N,NaN,NaN,NaN,Y,N,320,Moving Violation,2014-01-01,1044977,7:46,29,L,M,2014-01-01 07:46:00
3,NaN,NaN,N,NaN,NaN,NaN,Y,N,610,Moving Violation,2014-01-01,1044978,8:10,23,W,M,2014-01-01 08:10:00
4,NaN,NaN,N,NaN,NaN,NaN,N,N,930,Equipment Violation,2014-01-01,1044980,8:35,35,H,M,2014-01-01 08:35:00


In [6]:
sd_stop.service_area.value_counts()

310        32062
120        31712
520        30093
710        27279
240        25942
930        25142
110        24801
620        21446
230        20173
610        19507
720        17767
320        16808
430        16050
510        15232
810        14232
440        14022
830        13691
820        12605
Unknown    11446
530          667
630          203
130           90
840           29
Name: service_area, dtype: int64

In [7]:
sd_stop.subject_race.value_counts()

W    165602
H    119478
B     43687
O     27722
A     18159
F      5293
V      2937
C      2141
I      2026
X       645
K       566
J       485
P       434
Z       322
L       287
S       224
D       201
G       111
U        57
Name: subject_race, dtype: int64

In [8]:
# Load the Race Code Description
df_race_des = pd.read_csv('./vehicle_stops_race_codes.csv')
df_race_des.head()
df_race_des.rename(index=str, columns={'Race Code':'subject_race', 'Description': 'Race_Description'}, inplace=True)
#df_race_des.set_index('Race Code', inplace=True)
df_race_des.head()

,subject_race,Race_Description
0,A,OTHER ASIAN
1,B,BLACK
2,C,CHINESE
3,D,CAMBODIAN
4,F,FILIPINO


In [9]:
#Two ways to join and merge the data in dataframe
sd_stop = pd.merge(sd_stop, df_race_des, how='left', on='subject_race')
#sd_stop.join(other=df_race_des, on='subject_race')

In [10]:
sd_stop.subject_age.value_counts()

25     15882
30     14333
24     13404
23     13397
22     12616
26     12297
20     12161
21     12146
27     11577
28     11325
35     10655
32      9989
40      9979
29      9889
31      9715
34      8846
33      8811
45      8801
19      8387
50      7996
36      7822
37      7160
38      6980
43      6506
39      6308
42      6282
44      6273
41      5969
55      5959
46      5702
       ...  
228        1
55Q        1
F48        1
N          1
213        1
185        1
j          1
y          1
163        1
233        1
NN         1
3.7        1
221        1
211        1
212        1
456        1
5_         1
220        1
f28        1
111        1
`          1
f          1
120        1
119        1
114        1
204        1
f26        1
h          1
125        1
399        1
Name: subject_age, Length: 156, dtype: int64

When we check the subject age, it turns out some age datas are inserted non-correclty. It includes the following suspecious points:
1. Age is larger than 100
2. Age Typo

** These data is required to be cleanned before the further data analysis **

In [11]:
sd_stop.subject_sex.value_counts()

M    253397
F    136796
X        60
Name: subject_sex, dtype: int64

In [12]:
sd_stop[sd_stop.subject_sex == 'X']

,Unnamed: 15,Unnamed: as16,arrested,contraband_found,obtained_consent,property_seized,sd_resident,searched,service_area,stop_cause,stop_date,stop_id,stop_time,subject_age,subject_race,subject_sex,timestamp,Race_Description
3167,NaN,NaN,N,NaN,NaN,NaN,Y,N,610,"Muni, County, H&S Code",2014-01-16,1047579,2:00,20,O,X,2014-01-16 02:00:00,OTHER
9603,NaN,NaN,N,NaN,NaN,NaN,Y,N,520,Equipment Violation,2014-01-26,1053359,22:25,26,X,X,2014-01-26 22:25:00,NaN
44648,NaN,NaN,N,NaN,NaN,NaN,Y,N,610,Equipment Violation,2014-04-08,1088931,2:35,32,W,X,2014-04-08 02:35:00,WHITE
55670,NaN,NaN,N,NaN,NaN,NaN,Y,N,610,Moving Violation,2014-04-30,1100138,22:50,36,W,X,2014-04-30 22:50:00,WHITE
56050,NaN,NaN,N,NaN,NaN,NaN,N,N,510,Equipment Violation,2014-05-02,1100697,0:10,24,O,X,2014-05-02 00:10:00,OTHER
89287,NaN,NaN,N,NaN,NaN,NaN,Y,N,310,Moving Violation,2014-07-23,1133887,10:28,77,W,X,2014-07-23 10:28:00,WHITE
93594,NaN,NaN,N,NaN,NaN,NaN,Y,N,440,Equipment Violation,2014-08-05,1138774,3:40,24,W,X,2014-08-05 03:40:00,WHITE
93670,NaN,NaN,N,NaN,NaN,NaN,N,N,110,Moving Violation,2014-08-05,1139265,10:00,50,A,X,2014-08-05 10:00:00,OTHER ASIAN
93671,NaN,NaN,N,NaN,NaN,NaN,N,N,110,Moving Violation,2014-08-05,1139266,10:00,50,A,X,2014-08-05 10:00:00,OTHER ASIAN
94180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110,Moving Violation,2014-08-06,1139294,11:11,99,A,X,2014-08-06 11:11:00,OTHER ASIAN


Sex X looks something suspecious, but it is actually be entered into the database

In [13]:
sd_stop['stop_cause'].value_counts()

Moving Violation                      285562
Equipment Violation                    99577
Radio Call/Citizen Contact              1941
Muni, County, H&S Code                  1442
Personal Knowledge/Informant             884
Suspect Info (I.S., Bulletin, Log)       578
No Cause Specified on a Card             430
Personal Observ/Knowledge                104
UNI, &County, H&&S Code                   73
Other                                     39
MUNI, County, H&S Code                    29
&Moving Violation                         21
Suspect Info                              17
none listed                                6
not marked                                 4
&Equipment Violation                       3
NOT SPECIFIED                              2
NOTHING MARKED                             2
NOT MARKED                                 2
not noted                                  2
Bicycle                                    2
wm23                                       1
&Radio Cal

When the stop cause is checked, we find that some of the causes are 
1. with upper-case alphabets 
2. some alphabets are missed

The stop cause column can be further cleanned with the following code (Only focus on the major stop cause)

In [14]:
sd_stop.loc[sd_stop[sd_stop.stop_cause == '&Moving Violation'].index, 'stop_cause'] = 'Moving Violation'
sd_stop.loc[sd_stop[sd_stop.stop_cause == '&Equipment Violation'].index, 'stop_cause'] = 'Equipment Violation'
sd_stop.loc[sd_stop[sd_stop.stop_cause == '&Radio Call/Citizen Contact'].index, 'stop_cause'] = 'Radio Call/Citizen Contact'
sd_stop.loc[sd_stop[sd_stop.stop_cause == 'UNI, &County, H&&S Code'].index, 'stop_cause'] = 'Muni, County, H&S Code'
sd_stop.loc[sd_stop[sd_stop.stop_cause == 'MUNI, County, H&S Code'].index, 'stop_cause'] = 'Muni, County, H&S Code'

## Age-Stop Cause Relation Analysis
The procedure includes:
1. Data Clean
2. Data Visualization

### Clean Age Data

In [15]:
def isInt(anyNumberOrString):
    try:
        int(anyNumberOrString) #to check float and int use "float(anyNumberOrString)"
        return True
    except ValueError :
        return False

isInt("A") #False
isInt("5") #True
isInt(8) #True
isInt("5.88") #False *see comment above on how to make this True

False

In [16]:
int_list = [i for i, age in enumerate(sd_stop.subject_age) if isInt(age) == True]
sd_stop_age = sd_stop.iloc[int_list, :]

In [17]:
# Convert the string data to int data
sd_stop_age.loc[:,'subject_age'] = pd.to_numeric(sd_stop_age.subject_age, downcast='signed') 

/Users/macyeh/anaconda3/envs/ds/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [18]:
df_age = sd_stop_age[(sd_stop_age['subject_age'] > 15) & 
            (sd_stop_age['subject_age'] < 100)
           ] # Find the age between 15 and 100

### Clean Race Data